In [ ]:
#@title Imports
import os
import re
import tiktoken


# SimpleTokenzierV1

In [ ]:
# for item in os.listdir('.'): # '.' refers to the current directory
#     print(item)

os.getcwd()

In [ ]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
  raw_txt = f.read()

print(len(raw_txt))


In [ ]:
# Remove whitespace or not?
#   Removing whitespaces reduces memory and computing requirement
#   White spaces can be useful for text sensitive to the structure, like python indention
preprossed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_txt)
preprossed = [t.strip() for t in preprossed if t.strip()]
print(preprossed[:10])
print(len(preprossed))

In [ ]:
# Vocabulary: all the unique tokens in alphbetically order


# For tokens not in the vocab
UNKNOWN_TOKEN = "<|unk|>"

# Added between text sources.
# Allow the LLM to process and understand the data better.
END_OF_TEXT_TOKEN = "<|endoftext|>"

# Following special tokens are used by different types of tokenizers
# [BOS]: beginning of sequence
# [EOS]: end of sequence
# [PAD]: padding

sorted_unique_tokens = sorted(set(preprossed))
sorted_unique_tokens.extend([END_OF_TEXT_TOKEN, UNKNOWN_TOKEN])
print(len(sorted_unique_tokens))

In [ ]:
# Encode token to token id
vocab = {token:id for id,token in enumerate(sorted_unique_tokens)}

# for i, item in enumerate(vocab.items()):
#   print(i, item)
#   if i > 20:
#     break

In [ ]:
class SimpleTokenzierV1:
  def __init__(self, vocab):
    self.token_to_id = vocab
    self.id_to_token = {id:token for token,id in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [t.strip() for t in preprocessed if t.strip()]
    preprocessed = [t if t in self.token_to_id else UNKNOWN_TOKEN for t in preprocessed]
    return [self.token_to_id[t] for t in preprocessed]

  def decode(self, ids):
    tokens = [self.id_to_token[id] for id in ids]
    text = " ".join(tokens)
    return re.sub(r'\s+([,.?!"()\'])', r'\1', text)

In [ ]:
tokenizer_v1 = SimpleTokenzierV1(vocab)

In [ ]:
text1 = "how are you, jessica!"
text2 = "do you like tea?"
test_ids = tokenizer_v1.encode(" <|endoftext|> ".join((text1, text2)))
print(test_ids)
print(tokenizer_v1.decode(test_ids))

# BPE

In [ ]:
tokenizer_bpe = tiktoken.get_encoding('gpt2')


text1 = "how are you, jessica!"
text2 = "doyou like tea? 好 ofjweoifewjfoiewh"
test_ids = tokenizer_bpe.encode(" <|endoftext|> ".join((text1, text2)), allowed_special={END_OF_TEXT_TOKEN})
print(test_ids)
print(tokenizer_bpe.decode(test_ids))